In [1]:
import pandas as pd
import numpy as np
import pickle as pk

import pickle as pk
import xgboost as xgb
import matplotlib.pyplot as plt

from scipy.stats import gaussian_kde
import h2o
from h2o.automl import H2OAutoML

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle("../../data/df.pkl")

### IA Filter

In [3]:
df = df.loc[df.type_bool == True]
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN10005,"[[10.027000000001863, 4.996, 8.081], [33.94500...","[[0.015000000006693881, 0.2977, 2.206], [18.94...","[[0.0, 1.033, 1.325], [13.015000000006694, 0.4...","[[0.03500000000349246, 1.826, 3.029], [48.875,...","[0.0, 1.0381919191919462, 2.0763838383838924, ...","([10.027000000001863, 10.782565656567513, 11.5...","([0.015000000006693881, 1.0530404040470573, 2....","([0.0, 1.0281717171717606, 2.056343434343521, ...","([0.03500000000349246, 1.0222929292964054, 2.0...","[2.000004411755352, 2.000003547336676, 2.00000...",...,-26.554909,15.768490,16.436792,17.114795,-20.292951,-24.222566,-11.295562,Ia,True,1.08327
SN10007,"[[0.0, 6.266, 10.48], [10.105999999999767, 17....","[[0.03200000000651926, 7.161, 9.313], [2.01600...","[[2.0, 1.24, 2.429], [10.121000000006461, 20.0...","[[0.9770000000062282, 0.6586, 5.329], [15.9930...","[0.0, 1.5136565656566285, 3.027313131313257, 4...","([0.0, 1.5132626262626456, 3.0265252525252913,...","([0.03200000000651926, 1.5451717171782005, 3.0...","([2.0, 3.49314141414145, 4.9862828282829, 6.47...","([0.9770000000062282, 2.480787878794107, 3.984...","[11.341091517194192, 8.812204685737349, 6.9798...",...,-99.877031,21.670852,24.155364,-26.622112,-27.616185,-19.981432,49.299250,Ia,True,0.82331
SN100361,"[[6.957000000002154, 16.02, 23.92], [7.9690000...","[[0.023000000001047738, 4.903, 3.324], [6.9840...","[[0.0, 0.7103, 2.626], [5.9760000000023865, 5....","[[8.055000000000291, 2.567, 2.665], [15.035000...","[0.0, 1.3031111111111275, 2.606222222222255, 3...","([6.957000000002154, 8.15910101010318, 9.36120...","([0.023000000001047738, 1.315494949496019, 2.6...","([0.0, 1.3026767676768058, 2.6053535353536117,...","([8.055000000000291, 9.27674747474778, 10.4984...","[8.233549869515556, 9.464488874691614, 10.4511...",...,102.074783,-95.727930,4.861555,-29.749073,23.442440,5.441237,12.503390,Ia,True,0.90012
SN100471,"[[0.0, 3.79, 4.764], [11.969000000004598, 1.67...","[[0.023999999997613486, 2.202, 3.146], [3.0429...","[[0.008000000001629815, 0.4567, 2.129], [3.004...","[[0.046999999998661224, 6.92, 2.486], [3.07500...","[0.0, 1.4638181818181601, 2.9276363636363203, ...","([0.0, 1.3621414141414037, 2.7242828282828073,...","([0.023999999997613486, 1.4875757575733735, 2....","([0.008000000001629815, 1.3701818181834458, 2....","([0.046999999998661224, 1.3283737373723963, 2....","[5.63245418874034, 5.603687474301877, 5.594769...",...,-40.831628,28.629443,12.801879,-0.555542,13.065768,-21.776767,9.358561,Ia,True,0.92640
SN100490,"[[0.0, 0.01278, 2.833], [8.11699999999837, 1.7...","[[0.026999999994586688, 1.849, 5.434], [8.1449...","[[15.0, 0.4107, 4.054], [46.042999999997846, 6...","[[1.1679999999978463, 1.572, 6.291], [49.96899...","[0.0, 0.7267979797979933, 1.4535959595959866, ...","([0.0, 0.5955656565656131, 1.1911313131312262,...","([0.026999999994586688, 0.7435858585804729, 1....","([15.0, 15.565262626262582, 16.130525252525164...","([1.1679999999978463, 1.8829999999978817, 2.59...","[8.351889035545303, 8.47302530763294, 8.594179...",...,-36.484191,38.259352,-88.907842,76.656123,50.003481,-5.379145,-57.678266,Ia,True,0.73814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN98918,"[[0.0, 3.693, 2.686], [4.984000000004016, 18.7...","[[5.94100000000617, 6.403, 9.191], [13.1840000...","[[4.996000000006461, 8.241, 6.049], [12.141000...","[[0.027000000001862645, 3.092, 4.045], [5.0120...","[0.0, 1.3522323232323885, 2.704464646464777, 4...","([0.0, 1.3017676767676767, 2.6035353535353534,...","([5.94100000000617, 7.233222222228395, 8.52544...","([4.996000000006461, 6.297686868693313, 7.5993...","([0.027000000001862645, 1.2480808080826955, 2....","[16.23491946702271, 16.82763060047037, 17.3637...",...,-34.255294,-4.973908,11.915895,-56.738013,-27.269352,9.479893,7.803001,Ia,True,0.76083
SN99404,"[[0.0, 6.586, 10.48], [15

### Auto ML

In [4]:
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O_cluster_uptime:,59 mins 18 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 11 days
H2O_cluster_name:,H2O_from_python_fmoliveira_9xfnr2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.858 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [18]:
X = df.loc[:,'f1':'REDSHIFT_SPEC']
y = df.loc[:,'REDSHIFT_SPEC']

In [16]:
aml = H2OAutoML(max_models=20, seed=1)

In [17]:
aml.train(x=list(df.loc[:,'f1':'f20'].columns), y='REDSHIFT_SPEC', training_frame=h2o.H2OFrame(X.iloc[0:int(len(X)/2)]))

Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [28]:
lb = aml.leaderboard.as_data_frame(use_pandas=True)
lb

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_AllModels_AutoML_20201030_001821,0.008248,0.090821,0.008248,0.070062,0.052370
1,StackedEnsemble_BestOfFamily_AutoML_20201030_0...,0.008308,0.091150,0.008308,0.070309,0.052530
2,GBM_2_AutoML_20201030_001821,0.008508,0.092238,0.008508,0.071366,0.053290
3,GBM_3_AutoML_20201030_001821,0.008642,0.092964,0.008642,0.071624,0.053667
4,GBM_1_AutoML_20201030_001821,0.008789,0.093747,0.008789,0.072825,0.054354
5,GBM_4_AutoML_20201030_001821,0.008870,0.094179,0.008870,0.072302,0.054383
6,GBM_grid__1_AutoML_20201030_001821_model_2,0.009131,0.095554,0.009131,0.074759,0.055470
7,GBM_grid__1_AutoML_20201030_001821_model_1,0.009161,0.095716,0.009161,0.075084,0.055418
8,DRF_1_AutoML_20201030_001821,0.009223,0.096038,0.009223,0.074752,0.055537
9,XRT_1_AutoML_20201030_001821,0.009299,0.096434,0.009299,0.075213,0.055852


In [59]:
best_model_sav = h2o.save_model(aml.leader, path = "../../models/h2o_ML_model_leader")
best_model_sav

'/home/fmoliveira/Documents/Pessoal/Supernova_Identification_Article/models/h2o_ML_model_leader/StackedEnsemble_AllModels_AutoML_20201030_001821'

In [5]:
XG_model = h2o.get_model('XGBoost_grid__1_AutoML_20201030_001821_model_3')
XG_model

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_grid__1_AutoML_20201030_001821_model_3


Model Summary: 


,,number_of_trees
0,,46.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.003373586720996698
RMSE: 0.058082585350487784
MAE: 0.04447338371946615
RMSLE: 0.03321090662435424
Mean Residual Deviance: 0.003373586720996698

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 0.00948144983131941
RMSE: 0.09737273659150908
MAE: 0.07558090773687301
RMSLE: 0.05651633804467251
Mean Residual Deviance: 0.00948144983131941

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.07558188,0.003390451,0.07244671,0.07437462,0.07285851,0.08019055,0.07803898
1,mean_residual_deviance,0.009481487,6.9631025E-4,0.009496162,0.009350085,0.008514539,0.010470397,0.009576253
2,mse,0.009481487,6.9631025E-4,0.009496162,0.009350085,0.008514539,0.010470397,0.009576253
3,r2,0.81751895,0.021214705,0.8159167,0.8360778,0.8363323,0.7844753,0.8147926
4,residual_deviance,0.009481487,6.9631025E-4,0.009496162,0.009350085,0.008514539,0.010470397,0.009576253
5,rmse,0.097320326,0.003577879,0.09744825,0.09669583,0.09227426,0.102324955,0.09785833
6,rmsle,0.056489255,0.0019484252,0.05732115,0.056751605,0.053547654,0.058838625,0.055987243



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2020-10-30 01:19:17,5.349 sec,0.0,0.322885,0.275523,0.104255
1,,2020-10-30 01:19:17,5.366 sec,5.0,0.114410,0.090958,0.013090
2,,2020-10-30 01:19:17,5.378 sec,10.0,0.089327,0.070375,0.007979
3,,2020-10-30 01:19:17,5.392 sec,15.0,0.082815,0.064832,0.006858
4,,2020-10-30 01:19:17,5.405 sec,20.0,0.077441,0.060583,0.005997
5,,2020-10-30 01:19:17,5.420 sec,25.0,0.072252,0.056252,0.005220
6,,2020-10-30 01:19:17,5.436 sec,30.0,0.068597,0.053141,0.004706
7,,2020-10-30 01:19:17,5.454 sec,35.0,0.064344,0.049686,0.004140
8,,2020-10-30 01:19:17,5.474 sec,40.0,0.061682,0.047430,0.003805
9,,2020-10-30 01:19:17,5.493 sec,45.0,0.058746,0.045000,0.003451



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f1,106.150856,1.000000,0.672532
1,f10,6.741690,0.063510,0.042713
2,f5,5.842342,0.055038,0.037015
3,f19,4.329252,0.040784,0.027429
4,f2,3.762764,0.035447,0.023839
5,f3,3.666935,0.034545,0.023232
6,f4,2.975584,0.028032,0.018852
7,f15,2.902452,0.027343,0.018389
8,f13,2.738504,0.025798,0.017350
9,f18,2.436839,0.022956,0.015439


In [55]:
XG_model_sav = h2o.save_model(XG_model, path = "../../models")
XG_model_sav

'/home/fmoliveira/Documents/Pessoal/Supernova_Identification_Article/models/XGBoost_grid__1_AutoML_20201030_001821_model_3'

In [56]:
XG_model_load = h2o.load_model('/home/fmoliveira/Documents/Pessoal/Supernova_Identification_Article/models/XGBoost_grid__1_AutoML_20201030_001821_model_3')

### Extracting within h2o

In [7]:
from h2o.tree import H2OTree

In [13]:
ntrees = 46
list_of_trees = [H2OTree(model = XG_model, tree_number = t, tree_class = None) for t in range(ntrees)]

In [22]:
df_valid = X.iloc[int(len(X)/2):-1].loc[:,'f1':'f20']
hf = h2o.H2OFrame(df_valid)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
leaf_nodes = XG_model.predict_leaf_node_assignment(hf, type='Node_ID').as_data_frame()

In [28]:
# tree_predictions is the dataframe with predictions for all the 70 trees
tree_predictions = pd.DataFrame(columns=['T'+str(t+1)+'.C1' for t in range(ntrees)])
for t in range(ntrees):
    tr = list_of_trees[t]
    node_ids = np.array(tr.node_ids)
    treePred = lambda n: tr.predictions[np.where(node_ids==n)[0][0]] 
    tree_predictions['T'+str(t+1)+'.C1'] = leaf_nodes['T'+str(t+1)+'.C1'].apply(treePred)

In [37]:
tree_predictions.set_index(df_valid.index, inplace = True)
tree_predictions

,T1.C1,T2.C1,T3.C1,T4.C1,T5.C1,T6.C1,T7.C1,T8.C1,T9.C1,T10.C1,...,T37.C1,T38.C1,T39.C1,T40.C1,T41.C1,T42.C1,T43.C1,T44.C1,T45.C1,T46.C1
ID,,,,,,,,,,,,,,,,,,,,,
SN509093,0.073818,0.027184,0.019053,0.016947,0.007865,0.002244,0.007448,0.012195,0.011142,0.004835,...,0.000939,-0.003584,-0.000655,-0.003245,-0.004244,-0.004635,0.002928,0.005573,-0.001795,-0.004458
SN509279,0.032150,0.027184,0.019053,0.037185,0.004307,0.000288,0.000348,0.001180,0.002238,-0.006075,...,0.000490,0.000739,0.001541,0.002038,-0.001126,-0.004716,-0.001555,-0.002667,0.001665,0.002814
SN509387,0.009296,-0.002851,-0.003805,0.005689,0.004307,0.012240,0.000348,-0.006236,0.002238,0.004835,...,-0.000575,0.000739,0.001541,0.002038,0.003603,0.002046,-0.001555,-0.002667,0.001665,-0.003704
SN50950,0.073818,0.027853,0.027178,0.018369,-0.009423,0.002244,-0.006872,0.001180,0.002238,-0.006075,...,-0.006733,0.005031,-0.007958,0.002038,0.000626,-0.008511,-0.001505,0.002471,-0.012721,-0.001439
SN509630,0.120294,0.073402,0.027178,0.037185,0.024105,0.020302,0.010409,0.001180,0.008917,-0.006075,...,-0.006733,-0.002818,0.003316,-0.001729,0.009432,-0.003284,0.003997,-0.001672,-0.012721,0.003522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN98880,-0.022901,-0.024006,0.002170,0.000664,-0.013619,-0.000414,-0.007832,-0.014682,-0.009349,0.001554,...,-0.006733,0.005031,-0.007958,0.007357,0.000646,-0.008511,0.012847,0.002471,-0.005694,0.000248
SN98918,0.120294,0.073402,0.063494,0.037185,0.037381,0.020302,0.007448,0.001180,0.008917,0.001554,...,-0.001431,-0.002818,-0.003641,-0.001729,0.000646,0.002349,0.003997,-0.004813,-0.000597,0.004375
SN99404,0.009296,-0.011708,-0.040282,0.005689,-0.030031,-0.000414,-0.019589,-0.016388,0.013553,-0.006605,...,0.001841,0.005031,-0.003155,0.008400,0.000626,-0.003284,0.004866,0.011095,-0.006887,-0.001439


### Exporting

In [38]:
preds = XG_model.predict(h2o.H2OFrame(X.iloc[int(len(X)/2):-1].loc[:,'f1':'f20']))

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


In [48]:
df_full = df.iloc[int((len(df)/2)+1):]
df_full['zphot'] = preds.as_data_frame()['predict'].tolist()
df_full

/home/fmoliveira/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC,zphot
ID,,,,,,,,,,,,,,,,,,,,,
SN509279,"[[18.89900000000489, 0.2945, 4.365], [26.906000000002678, 0.5292, ...","[[0.008000000001629815, 7.418, 8.133], [32.824000000000524, 3.325,...","[[0.0, 3.166, 3.995], [5.867000000005646, 0.4342, 2.751], [18.9060...","[[18.93000000000029, 0.689, 5.952], [26.93800000000192, 0.7062, 3....","[0.0, 1.623303030303052, 3.246606060606104, 4.869909090909156, 6.4...","([18.89900000000489, 20.331404040408902, 21.763808080812915, 23.19...","([0.008000000001629815, 1.6212020202036719, 3.234404040405714, 4.8...","([0.0, 1.613202020202042, 3.226404040404084, 4.839606060606126, 6....","([18.93000000000029, 20.24135353535383, 21.55270707070737, 22.8640...","[20.927748638000686, 22.041179082094693, 22.99256666902695, 23.622...",...,49.884274,82.560061,-94.391054,-86.511354,26.779625,-60.881068,Ia,True,0.65538,0.712204
SN509387,"[[10.089999999996508, 1.817, 25.57], [37.91399999999703, 0.3081, 2...","[[2.0, 2.957, 4.31], [10.944999999999709, 28.04, 30.96], [31.90600...","[[0.0, 5.014, 7.842], [42.91000000000349, 24.52, 6.082], [44.07400...","[[15.976999999998952, 0.967, 6.541], [29.940999999998894, 4.27, 4....","[0.0, 1.594262626262648, 3.188525252525296, 4.7827878787879445, 6....","([10.089999999996508, 11.582141414137938, 13.074282828279367, 14.5...","([2.0, 3.49313131313131, 4.98626262626262, 6.479393939393931, 7.97...","([0.0, 1.594262626262648, 3.188525252525296, 4.7827878787879445, 6...","([15.976999999998952, 17.248141414140374, 18.519282828281796, 19.7...","[6.575579676585663, 4.2190314913182885, 2.5365843937407724, 1.6717...",...,135.784689,196.657130,-74.459661,-83.874633,25.921586,-63.886573,Ia,True,0.45100,0.604814
SN50950,"[[0.0, 3.267, 4.768], [2.997000000003027, 6.712, 2.592], [4.075000...","[[0.023999999997613486, 48.76, 3.284], [3.0429999999978463, 43.93,...","[[0.008000000001629815, 29.71, 2.211], [3.004000000000815, 24.42, ...","[[0.046999999998661224, 49.9, 2.656], [3.0750000000043656, 44.99, ...","[0.0, 0.8286464646464652, 1.6572929292929304, 2.4859393939393954, ...","([0.0, 0.8285252525252285, 1.657050505050457, 2.4855757575756856, ...","([0.023999999997613486, 0.8504646464623306, 1.6769292929270476, 2....","([0.008000000001629815, 0.8365656565672706, 1.6651313131329113, 2....","([0.046999999998661224, 0.8735454545441292, 1.7000909090895973, 2....","[8.898194799968902, 9.262242225808274, 9.571029149431784, 9.772770...",...,40.663644,76.159330,40.570551,12.182103,27.299124,10.842452,Ia,True,0.51480,0.543946
SN509630,"[[0.0, 12.41, 9.412], [1.9839999999967404, 1.688, 4.022], [9.95999...","[[0.014999999999417923, 31.1, 6.866], [2.006999999997788, 31.9, 4....","[[0.006999999997788109, 29.98, 7.202], [2.0, 25.49, 3.226], [10.10...","[[0.023000000001047738, 21.16, 7.932], [2.014999999999418, 24.8, 4...","[0.0, 1.1505252525251979, 2.3010505050503958, 3.4515757575755934, ...","([0.0, 1.1505252525251979, 2.3010505050503958, 3.4515757575755934,...","([0.014999999999417923, 1.0752525252519078, 2.1355050505043978, 3....","([0.006999999997788109, 1.1269090909068709, 2.2468181818159536, 3....","([0.023000000001047738, 1.0833333333343622, 2.1436666666676767, 3....","[16.000170811346788, 16.460013780148202, 16.847617852257752, 17.09...",...,-17.538851,32.414132,29.735733,16.959055,8.597079,-1.332496,Ia,True,0.68329,0.602473
SN509788,"[[0.0, 4.189, 4.765], [11.969000000004598, 7.001, 5.686], [13.0630...","[[0.023999999997613486, 34.89, 3.218], [3.0429999999978463, 28.51,...","[[0.008000000001629815, 14.92, 2.152], [3.004000000000815, 14.12, ...","[[0.046999999998661224, 26.82, 2.535], [3.0750000000043656, 20.04,...","[0.0, 1.1713636363636746, 2.342727272727349, 3.5140909090910237, 4...","([0.0, 1.1712121212121653, 2.3424242424243307, 3.513636363636496, ...","([0.023999999997613486, 1.15424242424004, 2.2844848484824665, 3.41...","([0.008000000001629815, 1.1792828282844798, 2.350565

In [53]:
tree_predictions = pd.concat([tree_predictions,df_full[['zphot', 'REDSHIFT_SPEC']]], axis = 1, join = 'inner')

In [62]:
pd.set_option("display.max_columns", 10)

In [63]:
tree_predictions

,T1.C1,T2.C1,T3.C1,T4.C1,T5.C1,...,T44.C1,T45.C1,T46.C1,zphot,REDSHIFT_SPEC
ID,,,,,,,,,,,
SN509279,0.032150,0.027184,0.019053,0.037185,0.004307,...,-0.002667,0.001665,0.002814,0.712204,0.65538
SN509387,0.009296,-0.002851,-0.003805,0.005689,0.004307,...,-0.002667,0.001665,-0.003704,0.604814,0.45100
SN50950,0.073818,0.027853,0.027178,0.018369,-0.009423,...,0.002471,-0.012721,-0.001439,0.543946,0.51480
SN509630,0.120294,0.073402,0.027178,0.037185,0.024105,...,-0.001672,-0.012721,0.003522,0.602473,0.68329
SN509788,0.120294,0.073402,0.048411,0.054611,0.024105,...,-0.001672,-0.012721,0.003522,0.812267,0.82267
...,...,...,...,...,...,...,...,...,...,...,...
SN98880,-0.022901,-0.024006,0.002170,0.000664,-0.013619,...,0.002471,-0.005694,0.000248,0.182560,0.30050
SN98918,0.120294,0.073402,0.063494,0.037185,0.037381,...,-0.004813,-0.000597,0.004375,0.322142,0.76083
SN99404,0.009296,-0.011708,-0.040282,0.005689,-0.030031,...,0.011095,-0.006887,-0.001439,0.816601,0.89529


In [57]:
tree_predictions.to_csv('tree_predictions.csv')
tree_predictions.to_pickle('tree_predictions.pkl')

In [52]:
from scipy.stats import norm

In [6]:
XG_model.tree_method

In [65]:
import matplotlib.pyplot as plt
from xgboost import plot_tree